In [1]:
# Download the challenge set files
from pycbc.frame import read_frame
import pylab
from pycbc.filter import resample_to_delta_t, highpass, matched_filter
from pycbc.psd import interpolate, inverse_spectrum_truncation
from pycbc.waveform import get_fd_waveform
from pycbc.vetoes import power_chisq
from pycbc.events.ranking import newsnr
import numpy
%matplotlib inline

In [2]:
# An example of how to read the data from these files:
file_name = "challenge3-2048.gwf"

#ifos = ["H1"]#, "L1"]
#for ifo in ifos:
#    ts_ifo = read_frame(file_name, f'{ifo}:CHALLENGE3)

In [3]:
data = {}
psd = {}

pylab.figure(figsize=[10, 5])

for ifo in ifos:
    # Read in and precondition the data
    #ts = m.strain(ifo).highpass_fir(15, 512)
    data[ifo] = resample_to_delta_t(ts_ifo, 1.0/2048).crop(2, 2)

    # Estimate the power spectral density of the data
    # This chooses to use 2s samples in the PSD estimate.
    # One should note that the tradeoff in segment length is that
    # resolving narrow lines becomes more difficult.
    p = data[ifo].psd(2)
    p = interpolate(p, data[ifo].delta_f)
    p = inverse_spectrum_truncation(p, int(2 * data[ifo].sample_rate), low_frequency_cutoff=15.0)
    psd[ifo] = p
    
    pylab.plot(psd[ifo].sample_frequencies, psd[ifo], label=ifo)

pylab.yscale('log')
pylab.xscale('log')
#pylab.ylim(1e-47, 1e-41)
pylab.xlim(20, 1024)
pylab.ylabel('$Strain^2 / Hz$')
pylab.xlabel('Frequency (Hz)')
pylab.grid()
pylab.legend()
pylab.show()

NameError: name 'ifos' is not defined

<Figure size 720x360 with 0 Axes>

In [ ]:
def challenge_matched_filter(file_name,mass):
    print("Looking at file {} with template mass {} M_sol".format(file_name,mass))
    channel_name = "H1:CHALLENGE3"
    start = 800
    end = 3000
    ts = read_frame(file_name, channel_name, start, end)
    ts = highpass(ts, 15.0)
    strain_ts = resample_to_delta_t(ts, 1.0/2048)
    conditioned_ts = strain_ts.crop(2, 2)
    psd_ts = conditioned_ts.psd(4)
    psd_ts = interpolate(psd_ts, conditioned_ts.delta_f)
    psd_ts = inverse_spectrum_truncation(psd_ts, int(4 * conditioned_ts.sample_rate),
    low_frequency_cutoff=15)
    hp_x, _ = get_fd_waveform(approximant="IMRPhenomD",
                             mass1=mass, mass2=mass,
                            f_lower=20.0, delta_f=conditioned_ts.delta_f)
    hp_x.resize(len(psd_ts))

    # For each observatory use this template to calculate the SNR time series
    snr_x = matched_filter(hp_x, conditioned_ts, psd=psd_ts, low_frequency_cutoff=20).crop(5, 4)

    pylab.figure(figsize=[14, 4])
    pylab.plot(snr_x.sample_times, abs(snr_x), label='H1')
    pylab.title('SNR Time Series')
    pylab.grid()
    #pylab.xlim(100,120)
    #pylab.ylim(0, 15)
    pylab.xlabel('Time (s)')
    pylab.ylabel('Signal-to-noise (SNR)')
    pylab.show()

    chisq = {}
    nbins = 26
    chisq_x = power_chisq(hp_x, conditioned_ts, nbins, psd_ts, low_frequency_cutoff=20.0)
    chisq_x = chisq_x.crop(5, 4)

    dof_x = nbins * 2 - 2
    chisq_x /= dof_x


    # The rho-hat term above is named "newsnr" here
    nsnr_x = newsnr(abs(snr_x), chisq_x)

    # Plot the new SNR timeseries
    pylab.figure(figsize=[14, 4])
    pylab.plot(snr_x.sample_times, nsnr_x, label='H1')
    pylab.title('NewSNR Timeseries')
    pylab.grid()
    #pylab.xlim(100,120)
    #pylab.ylim(0, 15)
    pylab.xlabel('Time (s)')
    pylab.ylabel('Re-weighted Signal-to-noise')
    pylab.show()
    peak = numpy.argmax(abs(nsnr_x))
    snrp = nsnr_x[peak]
    time = ts.sample_times[peak]

    print("We found a signal at {}s with SNR {}".format(time, 
                                                        abs(snrp)))

challenge_matched_filter(file_name, 10)